In [4]:
import argparse
import os
import shutil
import sys
import glob
import subprocess
import pandas as pd
import datetime
import hashlib 
import os
from lxml import etree as et
from itertools import islice
from datetime import datetime
import datetime as dt
from itertools import islice
import time

In [5]:
def flip_function(df):
    flip_data = df.T
    flip_data.columns=flip_data.iloc[0]
    flip_data.reset_index()
    flip_data=flip_data[1:]
    return flip_data

In [7]:
# direc = 'C:/Users/jmdoherty/OneDrive - University of Massachusetts/Desktop/Mac_BL_L0'
direc = os.getcwd()

xlsx = []
for i in os.listdir(direc):
        if i.endswith('.xlsx'):
            i_split = i.split('.')[0]
            list = xlsx.append(i_split)
        else:
            continue
            
for i in range(len(xlsx)):

    f = direc+'/'+xlsx[i] +'.xlsx'
    xle_f= direc+'/'+xlsx[i] + '.xle'
    csv_f= direc+'/'+xlsx[i] + '.csv'

    project_prep = pd.read_excel(f, sheet_name='Project')
    projectdf= pd.DataFrame(project_prep)
    CollectionMetaData=flip_function(projectdf)
    DataGranuleMembers = pd.read_excel(f, sheet_name='DataGranuleMembers')
    
    ### .xle file size/checksum/time ###
    md5_hash=hashlib.md5()
    file=open(xle_f,"rb")
    content=file.read()
    md5_hash.update(content)
    xle_digest=md5_hash.hexdigest()

    xle_file_size = os.path.getsize(xle_f)  

    ti_m=os.path.getmtime(xle_f)
    m_ti=time.ctime(ti_m)
    t_obj=time.strptime(m_ti)
    xle_T_stamp=time.strftime("%Y-%m-%dT%H:%M:%S.000000Z",t_obj)
    
    ### .csv file size/checksum/time ###
    md5_hash=hashlib.md5()
    file=open(csv_f,"rb")
    content=file.read()
    md5_hash.update(content)
    csv_digest=md5_hash.hexdigest()
    
    csv_file_size= os.path.getsize(csv_f)
    
    ti_m=os.path.getmtime(csv_f)
    m_ti=time.ctime(ti_m)
    t_obj=time.strptime(m_ti)
    csv_T_stamp=time.strftime("%Y-%m-%dT%H:%M:%S.000000Z",t_obj)

    #Need to be sure that this is in UTC time and NOT EST. I think it is getting the time based on the time from the local device, might beed to configure something on the cluster?
    current_time= str(datetime.now()).replace(" ","T")+("Z")

    root= et.Element('GranuleMetaDataFile')

    for index,row in CollectionMetaData.iterrows():
        Column_heading_1 = et.SubElement(root,'Project')
        Column_heading_2 = et.SubElement(root, 'FileIdentifier')

    root_tags=et.SubElement(root,'CollectionMetaData')

    for index,row in CollectionMetaData.iterrows():
        Column_heading_3 = et.SubElement(root_tags, 'Campaign')
        Column_heading_4 = et.SubElement(root_tags, 'CampaignShortName')
        Column_heading_5 = et.SubElement(root_tags, 'ShortName')
        Column_heading_6 = et.SubElement(root_tags, 'InstrumentName')
        Column_heading_7 = et.SubElement(root_tags, 'Description')

        Column_heading_1.text = str(row['Project'])
        Column_heading_2.text = str(row['FileIdentifier'])
        Column_heading_3.text = str(row['Campaign'])
        Column_heading_4.text = str(row['CampaignShortName'])
        Column_heading_5.text = str(row['ShortName'])
        Column_heading_6.text = str(row['InstrumentName'])
        Column_heading_7.text = str(row['Description'])

    root_tags = et.SubElement(root, 'Extent')
    root_tags1 = et.SubElement(root_tags,'TemporalExtent')
    root_tags2 = et.SubElement(root_tags,'SpatialExtent')

    for index,row in CollectionMetaData.iterrows():
        Column_heading_8 = et.SubElement(root_tags1, 'StartDateTime')
        Column_heading_9 = et.SubElement(root_tags1, 'EndDateTime')
        Column_heading_10 = et.SubElement(root_tags2, 'WestBoundLongitude')
        Column_heading_11 = et.SubElement(root_tags2, 'EastBoundLongitude')
        Column_heading_12 = et.SubElement(root_tags2, 'SouthBoundLatitude')
        Column_heading_13 = et.SubElement(root_tags2, 'NorthBoundLatitude')

        Column_heading_8.text = str(row['StartDateTime'])
        Column_heading_9.text = str(row['EndDateTime'])
        Column_heading_10.text = str(row['WestBoundLongitude'])
        Column_heading_11.text = str(row['EastBoundLongitude'])
        Column_heading_12.text = str(row['SouthBoundLatitude'])
        Column_heading_13.text = str(row['NorthBoundLatitude'])

    root_tags = et.SubElement(root, 'DataGranuleMembers')

    for index,row in CollectionMetaData.iterrows():
        root_tags1= et.SubElement(root_tags,'DataGranuleMember')
        Column_heading_14 = et.SubElement(root_tags1, 'LocalGranuleID')
        Column_heading_15 = et.SubElement(root_tags1, 'ProductionDateTime')

        Column_heading_14.text = str(row['LocalGranuleID'])
        Column_heading_15.text = str(current_time)

    for index,row in DataGranuleMembers.iterrows():
        if str("xle") in (row['LocalGranuleID']):
            root_tags1= et.SubElement(root_tags,'DataGranuleMember')
            Column_heading_16 = et.SubElement(root_tags1, 'LocalGranuleID')
            Column_heading_17 = et.SubElement(root_tags1, 'ProductionDateTime')
            Column_heading_18 = et.SubElement(root_tags1, 'Checksum')
            Column_heading_19 = et.SubElement(root_tags1, 'CheckSumType')
            Column_heading_20 = et.SubElement(root_tags1, 'Size')

            Column_heading_16.text = str(row['LocalGranuleID'])
            Column_heading_17.text = str(xle_T_stamp)
            Column_heading_18.text = str(xle_digest)
            Column_heading_19.text = str(row['CheckSumType'])
            Column_heading_20.text = str(xle_file_size)
        else:
            root_tags1= et.SubElement(root_tags,'DataGranuleMember')
            Column_heading_16 = et.SubElement(root_tags1, 'LocalGranuleID')
            Column_heading_17 = et.SubElement(root_tags1, 'ProductionDateTime')
            Column_heading_18 = et.SubElement(root_tags1, 'Checksum')
            Column_heading_19 = et.SubElement(root_tags1, 'CheckSumType')
            Column_heading_20 = et.SubElement(root_tags1, 'Size')

            Column_heading_16.text = str(row['LocalGranuleID'])
            Column_heading_17.text = str(csv_T_stamp)
            Column_heading_18.text = str(csv_digest)
            Column_heading_19.text = str(row['CheckSumType'])
            Column_heading_20.text = str(csv_file_size)

    root_tags = et.SubElement(root, 'Additional')

    for index,row in CollectionMetaData.iterrows():
        Column_heading_20 = et.SubElement(root_tags, 'Personnel')
        Column_heading_21 = et.SubElement(root_tags, 'Contact')
        Column_heading_22 = et.SubElement(root_tags, 'BLModel')
        Column_heading_23 = et.SubElement(root_tags, 'BLSerial')
        Column_heading_24 = et.SubElement(root_tags, 'Latitude')
        Column_heading_25 = et.SubElement(root_tags, 'Longitude')
        Column_heading_26 = et.SubElement(root_tags, 'Install_DateTime')
        Column_heading_27 = et.SubElement(root_tags, 'Uninstall_DateTime')

        Column_heading_20.text = str(row['Personnel'])
        Column_heading_21.text = str(row['Contact'])
        Column_heading_22.text = str(row['BLModel'])
        Column_heading_23.text = str(row['BLSerial'])
        Column_heading_24.text = str(row['Latitude'])
        Column_heading_25.text = str(row['Longitude'])
        Column_heading_26.text = str(row['Install_DateTime'])
        Column_heading_27.text = str(row['Uninstall_DateTime'])


    tree = et.ElementTree(root) 
    et.indent(tree, space="\t", level=0)
    tree.write(direc+'/'+xlsx[i]+".xml", encoding="utf-8")

print("Yeehaw!")

Yeehaw!
